In [1]:
import asyncio
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network
import pandas as pd
from pyinjective.client.model.pagination import PaginationOption
from functions import marketId_to_ticker,extract_eth_address,map_eth_to_inj,mapper

network = Network.local()
client = AsyncClient(network)


async def get_positions(client,market_ids):
    skip = 0
    limit = 100
    array = [market_ids]
    pagination = PaginationOption(skip=skip, limit=limit)
    response = await client.fetch_derivative_positions_v2(market_ids=array[0], pagination=pagination)
    positions = pd.DataFrame([order for order in response['positions']])
    return positions

ticker = "EUR/USDT PERP"

market_id = await marketId_to_ticker(ticker)
df = await get_positions(client,market_id)
df["entryPrice"] = df["entryPrice"].astype(float)/10**6
df["margin"] = df["margin"].astype(float)/10**6  
df["liquidationPrice"] = df["liquidationPrice"].astype(float)/10**6
df["markPrice"] = df["markPrice"].astype(float)/10**6

df["Dollars_size_$"] = df["quantity"].astype(float)*df["markPrice"]

columns_to_remove = ['marketId', 'updatedAt','updatedAt','denom']
df = df.drop(columns=columns_to_remove)

df['subaccountId'] = df['subaccountId'].apply(extract_eth_address)
eth_addresses_array = df['subaccountId'].unique()

eth_to_inj_dict = {}

await mapper(eth_addresses_array,eth_to_inj_dict)

df['subaccountId'] = df['subaccountId'].map(eth_to_inj_dict)

df["quantity"]=df["quantity"].astype(float)

df.sort_values("quantity",ascending=False,inplace=True)
df

/Users/rvie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,ticker,subaccountId,direction,quantity,entryPrice,margin,liquidationPrice,markPrice,Dollars_size_$
5,EUR/USDT PERP,inj14k7wgey85pt226ugr9sq4h2ya774n82hzy26nf,short,658661.2,1.036160,11315.483970,1.048099,1.03607,682419.109484
3,EUR/USDT PERP,inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z,long,652909.2,1.036160,9664.546258,1.026490,1.03607,676459.634844
2,EUR/USDT PERP,inj12amsuk3z7k8e6xz6sjht0hyuchxzx9533pz5e4,long,6337.7,1.030421,641.959154,0.933798,1.03607,6566.300839
1,EUR/USDT PERP,inj1xyc8au3vwlxgnnt7vyjxvz83twpdp8lde3qmkg,short,282.9,1.036940,15.740586,1.087144,1.03607,293.104203
0,EUR/USDT PERP,inj1zqp822amue7efz7maa0ljflukjql4d0y3c66ge,short,192.4,1.038800,5.270255,1.060888,1.03607,199.339868
4,EUR/USDT PERP,inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z,short,110.4,1.053100,2.564864,1.070978,1.03607,114.382128
